In [ ]:
import time, sys, os, random, logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyiota.iota
RING = pyiota.iota.run4

#l = logging.getLogger('pyiota.acnet.acsys')
#l.setLevel(logging.WARNING)

pyIOTA acnet module has a goal of making life easier for Python scripting while hiding ACNET 'features'. In some ways, this brings the interface closer to EPICS, however there are fundamental differences in how ACNET works (clock events, lack of device monitoring) that prevent a fully unified interface. 

General pyiota strategy is:
- parse provided channel names into DRF2
- combine channels into 'sets' that will be operated on in parallel - this is key for good performance
- provide convenience methods on sets for typical operations - read/write/on/off/etc.
- translate requests into internal commands through multiple 'adapters' - ACL (HTTP web backend), DIODMQ (through ACNET-Proxy project) and DPM (through modified ACSys)
- as far as possible, return results in a uniform format
- provide convenience functions like caching, periodic monitoring, and others

## DRF2

ACNET uses (for now) DRF2 format https://www-bd.fnal.gov/controls/public/drf2/. A subset of most common format options is implemented in acnet.drf2 submodule, mirroring the Java implementation. For all device operations, channel string must be parseable into a valid DRF2 on creation. In addition, DPM-style '<-' extra postfix is allowed for certain requests.

In [ ]:
from pyiota.acnet.drf2 import parse_request, DRF_PROPERTY, ARRAY_RANGE, ImmediateEvent

In [ ]:
drf2 = parse_request('Z:ACLTST')
print(drf2)

In [ ]:
drf2 = parse_request('N:I2B1RI[50:]@p,1000')
print(drf2)

In [ ]:
# You can convert back to string with replacement of each DRF2 part
# For example, this is how periodic/immediate events are swapped internally
drf2.to_canonical()

In [ ]:
drf2.to_canonical(property=DRF_PROPERTY.SETTING, range=ARRAY_RANGE(low=20, high=240), event=ImmediateEvent())

In [ ]:
drf2.to_qualified(property=DRF_PROPERTY.SETTING)

## Adapters

Adapters are objects that take Devices and operate on them to do actual readings and settings. For complicated reasons, each adapter is best suited to specific requests and incompatible with others. Their properties are described below:

In [ ]:
from pyiota.acnet import ACL, ACNETRelay, DPM, READ_METHOD

# Java proxy, which is our custom Java HTTP server (ACNET-Proxy repo) that forwards all requests using DIODMQ (RabbitMQ broker). It is the highest performance method for large data, and has been in use for several years.
proxy = ACNETRelay(verbose=True)

# Java proxy but it is allowed to return last cached result if available (and error out if not)
# There is an internal hardcoded list of common IOTA devices that are subscribed to 10Hz updates on startup (i.e. N:I2B1RI@p,100)
# Since any fresh reads have >=30ms latency, cached data (~50ms age on average) is often as reasonable to use as dedicated immediate reads with much less ACNET load
# In some of the method discussed below, you can also request a 'fresh cached' read, which will poll until new periodic reading comes in and IS STRONGLY RECOMMENDED
proxy_cached = ACNETRelay(verbose=True, read_method=READ_METHOD.CACHED)

# ACL adapter uses the HTTP interfaced to execute small ACL one-liners
# It can be used to perform some unique operations not suitable for other adapters
# as well as being a very robust backup for reads. Quite fast at scale of hundreds of devices per request, but slow for individual ones.
# No settings are possible through ACL
acl = ACL(verbose=True)

# DPM interface uses a modified version of ACsys library to work through data pool manager
# It changes some internal mechanics to better work with threading and synchronous tasks, hiding async formalism
# Quite good read performance but settings are restricted by 'role'.
dpm = DPM(verbose=True)

In [ ]:
# You can ping proxy to verify connectivity
proxy.ping()

In [ ]:
# Example raw ACL command
acl._raw_command("event_info/nml/last_time A8")

In [ ]:
# Wait until next event (this is poor man's @e,A8 event alternative, but actually used operationally)
acl._raw_command("wait/nml/event/timeout=2 A8")

In [ ]:
# As expected, another event happened
acl._raw_command("event_info/nml/last_time A8")

In [ ]:
# Find out which DPM we are connected to
dpm.ping()

## Devices

Basic pyiota control objects are associated with specific data types as well as property types

Currently implemented:
- DoubleDevice = ACNET reading and setting of floats and ints 
- StatusDevice = ACNET status devices
- ArrayDevice = ACNET reading and setting of float arrays [this distinction from length-1 channels mirrors Java API, there is no technical reason for it]

In [ ]:
# Designate default adapter - it will be used unless overriden
pyiota.acnet.frontends.AdapterManager.default_adapter = dpm

In [ ]:
from pyiota.acnet import DoubleDevice, StatusDevice, ArrayDevice
test_double = DoubleDevice('Z:ACLTST')

In [ ]:
test_double.read()

In [ ]:
test_double.read(adapter=acl)

In [ ]:
test_double.read(adapter=proxy)

In [ ]:
# Instead of values, full DataResponse objects can be returned
# This is especially useful for responses that error out, since response will contain error codes
test_double.read(full=True, adapter=proxy)

In [ ]:
test_status = StatusDevice('Z|ACLTST')
test_status.read()

In [ ]:
test_status.set('OFF',full=True,adapter=proxy)

In [ ]:
# Each read also updates internal device state, which can be queries with convenience methods
test_status.on

In [ ]:
test_status.ready

In [ ]:
test_status.read(adapter=proxy)

In [ ]:
# Errors will produce 'None' results 
test_array = ArrayDevice('Z:CACHE[:2]')
test_array.read()

In [ ]:
test_array.read(adapter=proxy)

In [ ]:
test_array = ArrayDevice('N:IBC1RH[:10]')
test_array.read()

In [ ]:
test_array.read(adapter=proxy)

In [ ]:
test_array = ArrayDevice('Z:ACLTST[0:3]')
test_array.read()

In [ ]:
# that is real device used in internal Java tests...
t = DoubleDevice("C:CRAP") 
t.read()

In [ ]:
t.read(adapter=proxy)

In [ ]:
# Setting work, but you MUST change the property appropriately 
# Lack of automatic conversion is a safety feature such that explicit READING and SETTING devices need to be created
test_double = DoubleDevice('Z:ACLTST')
test_double.set(5, adapter=proxy)

In [ ]:
# This now succeeeds
test_double = DoubleDevice('Z_ACLTST')
test_double.set(5, adapter=proxy)

In [ ]:
# Status devices pair with control devices in same way as READING/SETTING ones do, but because there is no confusion there is automatic conversion
test_control = StatusDevice('Z&ACLTST')
test_control.set('OFF',full=True,adapter=proxy)

In [ ]:
test_status.read()
test_status.on

## Device sets

Previous section actually lied - pyiota operates on 'device sets', with individual reads just creating a 1-device set in the background. Why? Because ACNET is slow, and tuned around having premade large lists of devices for batch operations. Instead of trying to hide this complexity behind 'read_many_devices' style methods, the choice of how to combine devices is left up to the user. There are tradeoffs in terms of reliability and performance - read and set operations will wait until all devices finish, and too large jobs will slow down processing.

In [ ]:
from pyiota.acnet import DoubleDeviceSet
test_double_2 = DoubleDevice('Z_CACHE')
test_double_3 = DoubleDevice('G_CHIPLC')
ds_double = DoubleDeviceSet(members=[test_double, test_double_2, test_double_3], adapter=proxy)

In [ ]:
ds_double.read()

In [ ]:
ds_double.set([1.0, 2.0, 3.0])